In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import lightgbm as lgb
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import and load dataset
df_train = pd.read_csv('train.csv')
train_dict = pd.read_csv('train_data_dict.csv')
df_test = pd.read_csv('test.csv')
samp = pd.read_csv('sample_submission_lfbv3c3.csv')

In [3]:
df_train.isnull().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             113
patientid                               0
City_Code_Patient                    4532
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

In [4]:
df_test.isnull().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                              35
patientid                               0
City_Code_Patient                    2157
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
dtype: int64

In [6]:
df_train['Stay'].unique()

array(['0-10', '41-50', '31-40', '11-20', '51-60', '21-30', '71-80',
       'More than 100 Days', '81-90', '61-70', '91-100'], dtype=object)

In [7]:
#Encoding Target / Label
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
df_train['Stay'] = le.fit_transform(df_train['Stay'])

In [8]:
df_train['train_flag'] = 1
df_test['train_flag'] = 0
df_test['Stay'] = None
print(df_train.shape, df_test.shape)

df_data = pd.concat((df_train, df_test))
print(df_data.shape)

(318438, 19) (137057, 19)
(455495, 19)


In [9]:
#Drop Case_id and patient id - too many unique value
df_data.drop(['case_id','patientid'],axis=1,inplace=True)

In [10]:
#Handling the missing values
df_data['Bed Grade'] = np.where(df_data['Bed Grade'].isna(),1.0,df_data['Bed Grade'])
df_data['City_Code_Patient'] = np.where(df_data['City_Code_Patient'].isna(),0.0,df_data['City_Code_Patient'])

In [11]:
#Changing data type from float to int
df_data['Bed Grade'] = df_data['Bed Grade'].astype(int)
df_data['City_Code_Patient'] = df_data['City_Code_Patient'].astype(int)
df_data['Admission_Deposit'] = df_data['Admission_Deposit'].astype(int)

In [12]:
#Combined Data Analysis
df_data.shape, df_data.columns

((455495, 17),
 Index(['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
        'Hospital_region_code', 'Available Extra Rooms in Hospital',
        'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
        'City_Code_Patient', 'Type of Admission', 'Severity of Illness',
        'Visitors with Patient', 'Age', 'Admission_Deposit', 'Stay',
        'train_flag'],
       dtype='object'))

In [13]:
#Listing Categorical Columns
cat=df_data.select_dtypes(['object']).columns
cat

Index(['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age',
       'Stay'],
      dtype='object')

In [14]:
#Removig the Target / Label Column -'Stay'
cat=cat[:-1]
cat

Index(['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness',
       'Age'],
      dtype='object')

In [15]:
df_data = pd.get_dummies(data=df_data,columns=['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness',
       'Age'])

In [16]:
df_data.shape, df_data.dtypes

((455495, 52),
 Hospital_code                         int64
 City_Code_Hospital                    int64
 Available Extra Rooms in Hospital     int64
 Bed Grade                             int64
 City_Code_Patient                     int64
 Visitors with Patient                 int64
 Admission_Deposit                     int64
 Stay                                 object
 train_flag                            int64
 Hospital_type_code_a                  uint8
 Hospital_type_code_b                  uint8
 Hospital_type_code_c                  uint8
 Hospital_type_code_d                  uint8
 Hospital_type_code_e                  uint8
 Hospital_type_code_f                  uint8
 Hospital_type_code_g                  uint8
 Hospital_region_code_X                uint8
 Hospital_region_code_Y                uint8
 Hospital_region_code_Z                uint8
 Department_TB & Chest disease         uint8
 Department_anesthesia                 uint8
 Department_gynecology                 u

In [18]:
#Recovering Train and Test Dataset
df_train, df_test = df_data[df_data.train_flag == 1], df_data[df_data.train_flag == 0]
df_train.drop(['train_flag'], inplace=True, axis=1)
df_test.drop(['train_flag'], inplace=True, axis=1)
df_test.drop(['Stay'], inplace=True, axis=1)

In [19]:
print(df_train.shape, df_test.shape)

(318438, 51) (137057, 50)


In [20]:
del df_data

In [21]:
df_train['Stay'] = df_train['Stay'].astype(int)

In [22]:
df_train, df_eval = train_test_split(df_train, test_size=0.30, random_state=42, shuffle=True, stratify=df_train['Stay'])

In [23]:
feature_cols = df_train.columns.tolist()
feature_cols.remove('Stay')
#feature_cols.remove('Admission_Deposit')
label_col = 'Stay'
print(feature_cols)

['Hospital_code', 'City_Code_Hospital', 'Available Extra Rooms in Hospital', 'Bed Grade', 'City_Code_Patient', 'Visitors with Patient', 'Admission_Deposit', 'Hospital_type_code_a', 'Hospital_type_code_b', 'Hospital_type_code_c', 'Hospital_type_code_d', 'Hospital_type_code_e', 'Hospital_type_code_f', 'Hospital_type_code_g', 'Hospital_region_code_X', 'Hospital_region_code_Y', 'Hospital_region_code_Z', 'Department_TB & Chest disease', 'Department_anesthesia', 'Department_gynecology', 'Department_radiotherapy', 'Department_surgery', 'Ward_Type_P', 'Ward_Type_Q', 'Ward_Type_R', 'Ward_Type_S', 'Ward_Type_T', 'Ward_Type_U', 'Ward_Facility_Code_A', 'Ward_Facility_Code_B', 'Ward_Facility_Code_C', 'Ward_Facility_Code_D', 'Ward_Facility_Code_E', 'Ward_Facility_Code_F', 'Type of Admission_Emergency', 'Type of Admission_Trauma', 'Type of Admission_Urgent', 'Severity of Illness_Extreme', 'Severity of Illness_Minor', 'Severity of Illness_Moderate', 'Age_0-10', 'Age_11-20', 'Age_21-30', 'Age_31-40', '

In [24]:
cat_cols = feature_cols
cat_cols.remove('Admission_Deposit')
cat_cols

['Hospital_code',
 'City_Code_Hospital',
 'Available Extra Rooms in Hospital',
 'Bed Grade',
 'City_Code_Patient',
 'Visitors with Patient',
 'Hospital_type_code_a',
 'Hospital_type_code_b',
 'Hospital_type_code_c',
 'Hospital_type_code_d',
 'Hospital_type_code_e',
 'Hospital_type_code_f',
 'Hospital_type_code_g',
 'Hospital_region_code_X',
 'Hospital_region_code_Y',
 'Hospital_region_code_Z',
 'Department_TB & Chest disease',
 'Department_anesthesia',
 'Department_gynecology',
 'Department_radiotherapy',
 'Department_surgery',
 'Ward_Type_P',
 'Ward_Type_Q',
 'Ward_Type_R',
 'Ward_Type_S',
 'Ward_Type_T',
 'Ward_Type_U',
 'Ward_Facility_Code_A',
 'Ward_Facility_Code_B',
 'Ward_Facility_Code_C',
 'Ward_Facility_Code_D',
 'Ward_Facility_Code_E',
 'Ward_Facility_Code_F',
 'Type of Admission_Emergency',
 'Type of Admission_Trauma',
 'Type of Admission_Urgent',
 'Severity of Illness_Extreme',
 'Severity of Illness_Minor',
 'Severity of Illness_Moderate',
 'Age_0-10',
 'Age_11-20',
 'Age_21

In [25]:
label_col, len(feature_cols), len(cat_cols)

('Stay', 49, 49)

In [39]:
params = {}
params['learning_rate'] = 0.06
params['max_depth'] = 50
params['n_estimators'] = 1000
params['objective'] = 'multiclass'
params['boosting_type'] = 'gbdt'
params['subsample'] = 0.7
params['random_state'] = 42
params['colsample_bytree']=0.7
params['min_data_in_leaf'] = 55
params['reg_alpha'] = 1.6
params['reg_lambda'] = 1.1

In [40]:
clf = lgb.LGBMClassifier(**params)
    
clf.fit(df_train[feature_cols], df_train[label_col], early_stopping_rounds=50, 
        eval_set=[(df_train[feature_cols], df_train[label_col]),(df_eval[feature_cols], df_eval[label_col])], 
        eval_metric='multi_error', verbose=True, categorical_feature=cat_cols)

eval_score = accuracy_score(df_eval[label_col], clf.predict(df_eval[feature_cols]))

print('Eval ACC: {}'.format(eval_score))

[1]	valid_0's multi_error: 0.725249	valid_0's multi_logloss: 1.88247	valid_1's multi_error: 0.725254	valid_1's multi_logloss: 1.88242
Training until validation scores don't improve for 50 rounds
[2]	valid_0's multi_error: 0.724034	valid_0's multi_logloss: 1.86097	valid_1's multi_error: 0.724197	valid_1's multi_logloss: 1.86094
[3]	valid_0's multi_error: 0.718843	valid_0's multi_logloss: 1.84159	valid_1's multi_error: 0.718932	valid_1's multi_logloss: 1.84159
[4]	valid_0's multi_error: 0.706737	valid_0's multi_logloss: 1.82966	valid_1's multi_error: 0.706182	valid_1's multi_logloss: 1.82965
[5]	valid_0's multi_error: 0.700959	valid_0's multi_logloss: 1.81365	valid_1's multi_error: 0.700467	valid_1's multi_logloss: 1.81369
[6]	valid_0's multi_error: 0.692063	valid_0's multi_logloss: 1.80033	valid_1's multi_error: 0.691747	valid_1's multi_logloss: 1.80024
[7]	valid_0's multi_error: 0.684017	valid_0's multi_logloss: 1.78807	valid_1's multi_error: 0.684095	valid_1's multi_logloss: 1.78798
[

[62]	valid_0's multi_error: 0.578835	valid_0's multi_logloss: 1.54963	valid_1's multi_error: 0.578602	valid_1's multi_logloss: 1.54957
[63]	valid_0's multi_error: 0.57869	valid_0's multi_logloss: 1.54811	valid_1's multi_error: 0.578319	valid_1's multi_logloss: 1.54803
[64]	valid_0's multi_error: 0.578555	valid_0's multi_logloss: 1.54696	valid_1's multi_error: 0.578152	valid_1's multi_logloss: 1.54689
[65]	valid_0's multi_error: 0.578464	valid_0's multi_logloss: 1.54544	valid_1's multi_error: 0.578298	valid_1's multi_logloss: 1.54537
[66]	valid_0's multi_error: 0.57837	valid_0's multi_logloss: 1.54407	valid_1's multi_error: 0.577848	valid_1's multi_logloss: 1.54401
[67]	valid_0's multi_error: 0.578288	valid_0's multi_logloss: 1.543	valid_1's multi_error: 0.577733	valid_1's multi_logloss: 1.54295
[68]	valid_0's multi_error: 0.578301	valid_0's multi_logloss: 1.54174	valid_1's multi_error: 0.577765	valid_1's multi_logloss: 1.54169
[69]	valid_0's multi_error: 0.578157	valid_0's multi_loglos

[123]	valid_0's multi_error: 0.572265	valid_0's multi_logloss: 1.50379	valid_1's multi_error: 0.571787	valid_1's multi_logloss: 1.50362
[124]	valid_0's multi_error: 0.572093	valid_0's multi_logloss: 1.50337	valid_1's multi_error: 0.571432	valid_1's multi_logloss: 1.50319
[125]	valid_0's multi_error: 0.572027	valid_0's multi_logloss: 1.50301	valid_1's multi_error: 0.571421	valid_1's multi_logloss: 1.50283
[126]	valid_0's multi_error: 0.571785	valid_0's multi_logloss: 1.50259	valid_1's multi_error: 0.571159	valid_1's multi_logloss: 1.5024
[127]	valid_0's multi_error: 0.571659	valid_0's multi_logloss: 1.50217	valid_1's multi_error: 0.571118	valid_1's multi_logloss: 1.50198
[128]	valid_0's multi_error: 0.571665	valid_0's multi_logloss: 1.50174	valid_1's multi_error: 0.571055	valid_1's multi_logloss: 1.50155
[129]	valid_0's multi_error: 0.571656	valid_0's multi_logloss: 1.50138	valid_1's multi_error: 0.570908	valid_1's multi_logloss: 1.50117
[130]	valid_0's multi_error: 0.57165	valid_0's mu

[184]	valid_0's multi_error: 0.567505	valid_0's multi_logloss: 1.48425	valid_1's multi_error: 0.566648	valid_1's multi_logloss: 1.48397
[185]	valid_0's multi_error: 0.567417	valid_0's multi_logloss: 1.48401	valid_1's multi_error: 0.566679	valid_1's multi_logloss: 1.48372
[186]	valid_0's multi_error: 0.567313	valid_0's multi_logloss: 1.48378	valid_1's multi_error: 0.566438	valid_1's multi_logloss: 1.48349
[187]	valid_0's multi_error: 0.567231	valid_0's multi_logloss: 1.4835	valid_1's multi_error: 0.566334	valid_1's multi_logloss: 1.48323
[188]	valid_0's multi_error: 0.567172	valid_0's multi_logloss: 1.48325	valid_1's multi_error: 0.566313	valid_1's multi_logloss: 1.48296
[189]	valid_0's multi_error: 0.566939	valid_0's multi_logloss: 1.483	valid_1's multi_error: 0.566156	valid_1's multi_logloss: 1.48272
[190]	valid_0's multi_error: 0.566817	valid_0's multi_logloss: 1.48275	valid_1's multi_error: 0.56603	valid_1's multi_logloss: 1.48248
[191]	valid_0's multi_error: 0.566839	valid_0's mult

[245]	valid_0's multi_error: 0.563579	valid_0's multi_logloss: 1.47125	valid_1's multi_error: 0.562942	valid_1's multi_logloss: 1.47097
[246]	valid_0's multi_error: 0.563475	valid_0's multi_logloss: 1.47105	valid_1's multi_error: 0.562848	valid_1's multi_logloss: 1.47076
[247]	valid_0's multi_error: 0.563337	valid_0's multi_logloss: 1.47088	valid_1's multi_error: 0.562827	valid_1's multi_logloss: 1.47058
[248]	valid_0's multi_error: 0.563227	valid_0's multi_logloss: 1.4707	valid_1's multi_error: 0.562691	valid_1's multi_logloss: 1.4704
[249]	valid_0's multi_error: 0.563133	valid_0's multi_logloss: 1.47052	valid_1's multi_error: 0.562524	valid_1's multi_logloss: 1.47022
[250]	valid_0's multi_error: 0.563048	valid_0's multi_logloss: 1.47033	valid_1's multi_error: 0.562461	valid_1's multi_logloss: 1.47002
[251]	valid_0's multi_error: 0.563001	valid_0's multi_logloss: 1.47017	valid_1's multi_error: 0.562367	valid_1's multi_logloss: 1.46985
[252]	valid_0's multi_error: 0.562998	valid_0's mu

...

[733]	valid_0's multi_error: 0.541741	valid_0's multi_logloss: 1.41164	valid_1's multi_error: 0.540363	valid_1's multi_logloss: 1.4114
[734]	valid_0's multi_error: 0.541732	valid_0's multi_logloss: 1.41153	valid_1's multi_error: 0.540332	valid_1's multi_logloss: 1.41128
[735]	valid_0's multi_error: 0.541719	valid_0's multi_logloss: 1.41145	valid_1's multi_error: 0.540395	valid_1's multi_logloss: 1.41121
[736]	valid_0's multi_error: 0.541726	valid_0's multi_logloss: 1.41135	valid_1's multi_error: 0.540447	valid_1's multi_logloss: 1.41111
[737]	valid_0's multi_error: 0.541685	valid_0's multi_logloss: 1.41126	valid_1's multi_error: 0.540405	valid_1's multi_logloss: 1.41103
[738]	valid_0's multi_error: 0.541625	valid_0's multi_logloss: 1.41115	valid_1's multi_error: 0.540343	valid_1's multi_logloss: 1.41091
[739]	valid_0's multi_error: 0.541619	valid_0's multi_logloss: 1.41104	valid_1's multi_error: 0.540384	valid_1's multi_logloss: 1.41081
[740]	valid_0's multi_error: 0.541521	valid_0's m

[794]	valid_0's multi_error: 0.539128	valid_0's multi_logloss: 1.40561	valid_1's multi_error: 0.537841	valid_1's multi_logloss: 1.40536
[795]	valid_0's multi_error: 0.539107	valid_0's multi_logloss: 1.40551	valid_1's multi_error: 0.537862	valid_1's multi_logloss: 1.40525
[796]	valid_0's multi_error: 0.539122	valid_0's multi_logloss: 1.40542	valid_1's multi_error: 0.537862	valid_1's multi_logloss: 1.40516
[797]	valid_0's multi_error: 0.539075	valid_0's multi_logloss: 1.40534	valid_1's multi_error: 0.537841	valid_1's multi_logloss: 1.40507
[798]	valid_0's multi_error: 0.53905	valid_0's multi_logloss: 1.40525	valid_1's multi_error: 0.537767	valid_1's multi_logloss: 1.40498
[799]	valid_0's multi_error: 0.539022	valid_0's multi_logloss: 1.40514	valid_1's multi_error: 0.537705	valid_1's multi_logloss: 1.40487
[800]	valid_0's multi_error: 0.538971	valid_0's multi_logloss: 1.40506	valid_1's multi_error: 0.537673	valid_1's multi_logloss: 1.40479
[801]	valid_0's multi_error: 0.538978	valid_0's m

[855]	valid_0's multi_error: 0.537188	valid_0's multi_logloss: 1.39992	valid_1's multi_error: 0.535936	valid_1's multi_logloss: 1.39961
[856]	valid_0's multi_error: 0.537185	valid_0's multi_logloss: 1.39983	valid_1's multi_error: 0.535977	valid_1's multi_logloss: 1.39952
[857]	valid_0's multi_error: 0.537141	valid_0's multi_logloss: 1.39975	valid_1's multi_error: 0.535967	valid_1's multi_logloss: 1.39944
[858]	valid_0's multi_error: 0.537106	valid_0's multi_logloss: 1.39966	valid_1's multi_error: 0.535946	valid_1's multi_logloss: 1.39935
[859]	valid_0's multi_error: 0.537056	valid_0's multi_logloss: 1.39957	valid_1's multi_error: 0.535862	valid_1's multi_logloss: 1.39926
[860]	valid_0's multi_error: 0.537031	valid_0's multi_logloss: 1.39946	valid_1's multi_error: 0.535915	valid_1's multi_logloss: 1.39916
[861]	valid_0's multi_error: 0.537034	valid_0's multi_logloss: 1.39937	valid_1's multi_error: 0.535915	valid_1's multi_logloss: 1.39906
[862]	valid_0's multi_error: 0.537028	valid_0's 

[916]	valid_0's multi_error: 0.534776	valid_0's multi_logloss: 1.3944	valid_1's multi_error: 0.533371	valid_1's multi_logloss: 1.3941
[917]	valid_0's multi_error: 0.53476	valid_0's multi_logloss: 1.39433	valid_1's multi_error: 0.533371	valid_1's multi_logloss: 1.39402
[918]	valid_0's multi_error: 0.534729	valid_0's multi_logloss: 1.39423	valid_1's multi_error: 0.533361	valid_1's multi_logloss: 1.39391
[919]	valid_0's multi_error: 0.534697	valid_0's multi_logloss: 1.39414	valid_1's multi_error: 0.53334	valid_1's multi_logloss: 1.39383
[920]	valid_0's multi_error: 0.534672	valid_0's multi_logloss: 1.39406	valid_1's multi_error: 0.533298	valid_1's multi_logloss: 1.39375
[921]	valid_0's multi_error: 0.534644	valid_0's multi_logloss: 1.39399	valid_1's multi_error: 0.533277	valid_1's multi_logloss: 1.39367
[922]	valid_0's multi_error: 0.5346	valid_0's multi_logloss: 1.39389	valid_1's multi_error: 0.533266	valid_1's multi_logloss: 1.39358
[923]	valid_0's multi_error: 0.5346	valid_0's multi_lo

[977]	valid_0's multi_error: 0.532842	valid_0's multi_logloss: 1.38929	valid_1's multi_error: 0.531382	valid_1's multi_logloss: 1.38903
[978]	valid_0's multi_error: 0.532794	valid_0's multi_logloss: 1.38919	valid_1's multi_error: 0.531298	valid_1's multi_logloss: 1.38893
[979]	valid_0's multi_error: 0.532794	valid_0's multi_logloss: 1.38912	valid_1's multi_error: 0.531298	valid_1's multi_logloss: 1.38886
[980]	valid_0's multi_error: 0.532779	valid_0's multi_logloss: 1.38903	valid_1's multi_error: 0.53133	valid_1's multi_logloss: 1.38878
[981]	valid_0's multi_error: 0.532766	valid_0's multi_logloss: 1.38894	valid_1's multi_error: 0.531298	valid_1's multi_logloss: 1.38868
[982]	valid_0's multi_error: 0.532729	valid_0's multi_logloss: 1.38885	valid_1's multi_error: 0.531257	valid_1's multi_logloss: 1.38859
[983]	valid_0's multi_error: 0.532669	valid_0's multi_logloss: 1.38878	valid_1's multi_error: 0.531288	valid_1's multi_logloss: 1.38851
[984]	valid_0's multi_error: 0.532631	valid_0's m

In [41]:
best_iter = clf.best_iteration_
params['n_estimators'] = best_iter
print(params)

{'learning_rate': 0.06, 'max_depth': 50, 'n_estimators': 1000, 'objective': 'multiclass', 'boosting_type': 'gbdt', 'subsample': 0.7, 'random_state': 42, 'colsample_bytree': 0.7, 'min_data_in_leaf': 55, 'reg_alpha': 1.6, 'reg_lambda': 1.1}


In [48]:
df_train1 = pd.concat((df_train, df_eval))

In [49]:
clf1 = lgb.LGBMClassifier(**params)

clf1.fit(df_train1[feature_cols], df_train1[label_col], eval_metric='multi_error', 
        verbose=True, categorical_feature=cat_cols)

eval_score_acc = accuracy_score(df_train1[label_col], clf.predict(df_train1[feature_cols]))

print('ACC: {}'.format(eval_score_acc))

ACC: 0.4682851414353697


In [50]:
preds = clf.predict(df_test[feature_cols])
preds

array([2, 5, 2, ..., 1, 1, 5])

In [51]:
preds = le.inverse_transform(preds)
preds

array(['21-30', '51-60', '21-30', ..., '11-20', '11-20', '51-60'],
      dtype=object)

In [52]:
test1 = pd.read_csv('test.csv')

In [53]:
submit = pd.DataFrame({'case_id': test1.case_id, 'Stay': preds})
submit.head()



,case_id,Stay
0,318439,21-30
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60


In [54]:
submit.to_csv('Healcare2LightGBM2.csv', index=False)